# Sentiment Analysis

In [1]:
from __future__ import unicode_literals
import re, codecs
from collections import Counter
from nltk.tokenize.api import TokenizerI
import sys
from os import path
import codecs
import itertools
import fasttext
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
import re
import numpy as np
from sklearn.metrics import confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
from hazm import *

## Machine Learning Approach

In [2]:
df = pd.read_csv(r'Sentiment.txt',sep='\t')

df

,Sentiment,Text
0,__label__Neg,یکی از پرسنل شعبه به نام مراتی به علت خصومت شخ...
1,__label__Neg,وضعیت اسفبار ووحشتناک نوبت دندانپزشکی مخصوصا پ...
2,__label__Neg,متاسفانه باجه ای که منو ارجاع دادید مرخصی رفته...
3,__label__Neg,متاسفانه کارکنان مربوطه با اخلاق بد،بی حوصلگی ...
4,__label__Neg,خانم اربابی مزخرف
...,...,...
713,__label__Pos,کاش روند الکترونیکی کردن برای همه خدمات انجام بشه
714,__label__Pos,اما بقیه پرسنل خیلی کمک و راهنمایی می کردند
715,__label__Pos,عالییییییی
716,__label__Pos,واحد درآمد، سرعت عمل بیشتری داشته باشد


In [40]:
def clean(text,normalizer = Normalizer()):
    import itertools
    text = normalizer.normalize(text)
    text = ''.join(ch for ch, _ in itertools.groupby(text))
    return text.strip()


In [41]:
#!pip install pandarallel
from pandarallel import pandarallel


In [45]:
pandarallel.initialize(nb_workers=4)
result_parallel = df['Text'].parallel_apply(clean) 


INFO: Pandarallel will run on 100 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


Exception in thread Thread-105 (_handle_workers):
Traceback (most recent call last):
  File "C:\Users\alire\.pyenv\pyenv-win\versions\3.11.3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\NLP\NLP-P\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\alire\.pyenv\pyenv-win\versions\3.11.3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\alire\.pyenv\pyenv-win\versions\3.11.3\Lib\multiprocessing\pool.py", line 522, in _handle_workers
    cls._wait_for_updates(current_sentinels, change_notifier)
  File "C:\Users\alire\.pyenv\pyenv-win\versions\3.11.3\Lib\multiprocessing\pool.py", line 502, in _wait_for_updates
    wait(sentinels, timeout=timeout)
  File "C:\Users\alire\.pyenv\pyenv-win\versions\3.11.3\Lib\multiprocessing\connection.py", line 878, in wait
    ready_handles = _exhaustive_wait(waithandle_to_obj.keys(), timeout)
                 

In [44]:
result_parallel

0      یکی از پرسنل شعبه به نام مراتی به علت خصومت شخ...
1      وضعیت اسفبار وحشتناک نوبت دندانپزشکی مخصوصا پر...
2      متاسفانه باجه‌ای که منو ارجاع دادید مرخصی رفته...
3      متاسفانه کارکنان مربوطه با اخلاق بد، بی‌حوصلگی...
4                                      خانم اربابی مزخرف
                             ...                        
713    کاش روند الکترونیکی کردن برای همه خدمات انجام بشه
714           اما بقیه پرسنل خیلی کمک و راهنمای می‌کردند
715                                                 عالی
716               واحد درآمد، سرعت عمل بیشتری داشته باشد
717    به نظر من ایده خیلی خوبی بوده برای انجام کارها...
Name: Text, Length: 718, dtype: object

In [ ]:
train, test = train_test_split(df, test_size=0.2)
train_file = r'train.csv'
test_file = r'test.csv'
train.to_csv(train_file, sep="\t", quoting=csv.QUOTE_NONE, index=False, header=False,encoding='utf-8')
test.to_csv(test_file, sep="\t", quoting=csv.QUOTE_NONE, index=False, header=False,encoding='utf-8')

In [ ]:
su_model = fasttext.train_supervised(input=train_file, epoch=100, lr=1.0, wordNgrams=5, minCount=1,dim =200)
fasttext.train_unsupervised
su_model

In [ ]:
su_model.save_model(r'FasttextModel2.bin')

In [ ]:
su_model.get_word_vector('شعبه')

array([-3.23364548e-02,  6.29125535e-02, -9.96934436e-03, -2.33071409e-02,
       -3.34123112e-02,  5.84273450e-02,  8.80463747e-04,  7.42533337e-03,
        1.03353467e-02, -2.29718499e-02, -3.08449119e-02,  5.38741350e-02,
        5.10818958e-02, -1.99292600e-02, -5.97235840e-03,  3.05323116e-02,
       -1.38158230e-02, -3.97902653e-02,  7.95101076e-02, -1.26415312e-01,
        3.40842865e-02,  1.91603904e-03,  6.72887787e-02, -1.02161743e-01,
       -6.00176351e-03, -2.00359896e-02, -1.38403490e-01, -1.82072213e-03,
        2.44435612e-02, -4.53470647e-02, -5.74690364e-02,  5.58250025e-03,
       -5.53023666e-02, -6.31698743e-02, -6.21777512e-02, -7.59113356e-02,
        4.43873443e-02, -1.32729318e-02,  6.71401173e-02, -5.70142493e-02,
        5.04673943e-02, -6.41205814e-04,  4.66601029e-02, -9.37947445e-03,
        6.07297719e-02, -3.83471027e-02, -5.25359102e-02,  6.82672346e-03,
        7.44590089e-02,  5.36474586e-02,  7.71061853e-02, -1.84212178e-02,
       -6.88334554e-02,  

In [ ]:
su_model.get_sentence_vector('کارمندان شعبه خوب بودند')

array([-0.01311496,  0.02703333, -0.00483818, -0.01132382, -0.01159886,
        0.02259956, -0.00047575,  0.00251272,  0.00280101, -0.00857863,
       -0.01239226,  0.02030926,  0.0190504 , -0.00838206, -0.00462357,
        0.00897603, -0.00635855, -0.01436945,  0.02981447, -0.04782717,
        0.0143602 ,  0.00203382,  0.02551993, -0.04135893, -0.00291965,
       -0.00759567, -0.05447718, -0.00046405,  0.00970153, -0.01576062,
       -0.02225106,  0.00291018, -0.02029461, -0.02422708, -0.02281082,
       -0.02789188,  0.01615327, -0.00551164,  0.02455725, -0.02448724,
        0.02056898,  0.00067228,  0.01795144, -0.00468463,  0.0215255 ,
       -0.0163596 , -0.01958493,  0.00246451,  0.02784727,  0.01911625,
        0.0293433 , -0.00591376, -0.02640825,  0.00855471,  0.01057095,
       -0.03459552, -0.00484018, -0.01568855,  0.01596297,  0.00167147,
        0.01535848, -0.02766239, -0.02292398, -0.00868545,  0.01289462,
        0.00321713, -0.02247933, -0.00409247, -0.00223933,  0.02

In [ ]:
su_model.predict('کارمندان خیلی عالی بودند')

(('__label__Pos',), array([1.00000989]))

In [ ]:
su_model.predict(' شعبه صاحب ')

(('__label__Pos',), array([1.00001001]))

In [ ]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

In [ ]:
print_results(*su_model.test(test_file))

N	144
P@1	0.819
R@1	0.819


In [ ]:
def get_true_postive_count(cm, model, df, label_column_name, text_column_name, target_class):
    predictions = [model.predict(x)[0][0] for x in df[df[label_column_name] == target_class][text_column_name]]
    tp_count = len(list(filter(lambda x: x==target_class, predictions)))
    index = np.where(cm==tp_count)
    if index[0][0] == index[1][0]:
        return index[0][0]
    else:
        raise ValueError('COuld not find the index ' + index)
        
        
def resolve_labels_sequence(classes, cm, model, df, label_column_name, text_column_name):
    target_seq = [0] * len(classes)
    for label in classes:
        index = get_true_postive_count(cm, model, df, label_column_name, text_column_name, label)
        target_seq[index] = label
    return target_seq


def fasttext_confusion_matrix(model, pd_test_data, label_column_name, text_column_name):
    test_labels = pd_test_data[label_column_name]
    test_labels = np.array(test_labels)
    _classes = list(set(test_labels))
    pred_labels = [model.predict(x)[0][0] for x in pd_test_data[text_column_name]]
    pred_labels = np.array(pred_labels)
    eq = test_labels == pred_labels
    print("Accuracy: " + str(eq.sum() / len(test_labels)))
    cm = confusion_matrix(test_labels, pred_labels)
    labels = resolve_labels_sequence(_classes, cm, model, pd_test_data, label_column_name, text_column_name)
    print(labels)
    print(confusion_matrix(test_labels, pred_labels, labels=labels))
    df_cm = pd.DataFrame(cm, index = labels, columns = labels)
    plt.figure(figsize = (10,7))


In [ ]:
fasttext_confusion_matrix(su_model, test, 'Sentiment', 'Text')

Accuracy: 0.8194444444444444
['__label__Neg', '__label__Pos']
[[77  7]
 [19 41]]
